# 01. 행렬 Gradient 연산 심화

## 목표
- 행렬 미분에서 **전치(transpose)**가 어디에 붙는지 완벽하게 이해
- 벡터/행렬 미분의 핵심 패턴 학습
- ML에서 자주 나오는 gradient 계산 마스터

---

## 1. 기초: 왜 전치가 나타나는가?

### 1.1 기본 규칙

**핵심 원칙**: Gradient는 항상 **원래 변수와 같은 shape**을 가져야 한다!

- $x \in \mathbb{R}^{n}$ (column vector)이면 $\frac{\partial f}{\partial x} \in \mathbb{R}^{n}$ (column vector)
- $A \in \mathbb{R}^{m \times n}$이면 $\frac{\partial f}{\partial A} \in \mathbb{R}^{m \times n}$

### 1.2 Numerator Layout vs Denominator Layout

우리는 **Denominator Layout**을 사용합니다 (ML에서 표준):
- $\frac{\partial (Ax)}{\partial x} = A^T$ ← 전치가 붙는다!
- $\frac{\partial (x^T A)}{\partial x} = A$ ← 전치 없음!

> **왜?** Chain rule과 차원 맞추기 때문입니다. 아래에서 자세히 설명합니다.

## 2. 핵심 공식들

### 2.1 선형 변환

#### 공식 1: $\frac{\partial (Ax)}{\partial x} = A^T$

**유도:**
- $y = Ax$라고 하자. $y \in \mathbb{R}^m, x \in \mathbb{R}^n, A \in \mathbb{R}^{m \times n}$
- $y_i = \sum_{j=1}^{n} A_{ij} x_j$
- $\frac{\partial y_i}{\partial x_k} = A_{ik}$
- 따라서 Jacobian은: $J = \begin{bmatrix} \frac{\partial y_1}{\partial x_1} & \cdots & \frac{\partial y_1}{\partial x_n} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_m}{\partial x_1} & \cdots & \frac{\partial y_m}{\partial x_n} \end{bmatrix} = A$

하지만 우리가 원하는 것은 scalar function $f(y)$의 $x$에 대한 gradient:

$$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial y} \frac{\partial y}{\partial x} = \frac{\partial f}{\partial y} J^T = \frac{\partial f}{\partial y} A^T$$

만약 $f(y) = y$ (identity)라면:
$$\frac{\partial (Ax)}{\partial x} = A^T$$

#### 공식 2: $\frac{\partial (x^T A)}{\partial x} = A$

**유도:**
- $z = x^T A$, $z \in \mathbb{R}^{1 \times m}$ (row vector)
- $z_j = \sum_{i=1}^{n} x_i A_{ij}$
- $\frac{\partial z_j}{\partial x_k} = A_{kj}$
- 따라서: $\frac{\partial (x^T A)}{\partial x} = A$ (전치 없음!)

In [ ]:
import numpy as np

# 공식 검증: d(Ax)/dx = A^T
np.random.seed(42)

# A: 3x2 행렬, x: 2x1 벡터
A = np.array([[1, 2],
              [3, 4],
              [5, 6]])
x = np.array([[1],
              [2]])

# Ax 계산
y = A @ x
print("A shape:", A.shape)
print("x shape:", x.shape)
print("Ax shape:", y.shape)
print("\nAx =", y.T)

# Gradient는 A^T
print("\nGradient d(Ax)/dx = A^T:")
print(A.T)
print("Shape:", A.T.shape, "← x와 같은 shape (2x1)")

### 2.2 이차 형식 (Quadratic Forms)

#### 공식 3: $\frac{\partial (x^T A x)}{\partial x} = (A + A^T)x$

**유도:**
- $f(x) = x^T A x = \sum_{i,j} x_i A_{ij} x_j$
- $\frac{\partial f}{\partial x_k} = \sum_j A_{kj} x_j + \sum_i x_i A_{ik}$
- $= (Ax)_k + (A^T x)_k$
- $= [(A + A^T)x]_k$

**특별한 경우: A가 대칭행렬**
- $A = A^T$이면: $\frac{\partial (x^T A x)}{\partial x} = 2Ax$

In [ ]:
# 공식 검증: d(x^T A x)/dx

# 대칭 행렬
A_sym = np.array([[2, 1],
                   [1, 3]])
x = np.array([[1],
              [2]])

# x^T A x 계산
quad_form = x.T @ A_sym @ x
print("x^T A x =", quad_form[0, 0])

# Gradient (대칭이므로 2Ax)
grad = 2 * A_sym @ x
print("\nGradient d(x^T A x)/dx = 2Ax:")
print(grad.T)

# 비대칭 행렬인 경우
A_nonsym = np.array([[1, 2],
                      [3, 4]])
quad_form2 = x.T @ A_nonsym @ x
print("\n\n비대칭 행렬 경우:")
print("x^T A x =", quad_form2[0, 0])

grad2 = (A_nonsym + A_nonsym.T) @ x
print("Gradient d(x^T A x)/dx = (A + A^T)x:")
print(grad2.T)

### 2.3 선형 항 + 이차 항

#### 공식 4: $\frac{\partial (b^T x + x^T A x)}{\partial x} = b + (A + A^T)x$

이것은 **Linear Regression의 gradient**입니다!

**예제: MSE Loss**

$$L(w) = \frac{1}{2}||Xw - y||^2 = \frac{1}{2}(Xw - y)^T(Xw - y)$$

전개하면:
$$L(w) = \frac{1}{2}w^T X^T X w - w^T X^T y + \frac{1}{2}y^T y$$

Gradient:
$$\frac{\partial L}{\partial w} = X^T X w - X^T y$$

최적해 (gradient = 0):
$$w^* = (X^T X)^{-1} X^T y$$

In [ ]:
# Linear Regression gradient 유도 검증

# 데이터 생성
np.random.seed(42)
n_samples = 100
n_features = 3

X = np.random.randn(n_samples, n_features)
w_true = np.array([[2], [3], [-1]])
y = X @ w_true + np.random.randn(n_samples, 1) * 0.1

print("X shape:", X.shape)
print("y shape:", y.shape)

# Gradient 계산
def compute_gradient(X, y, w):
    """MSE loss의 gradient"""
    return X.T @ (X @ w - y)

# 초기 w
w_init = np.zeros((n_features, 1))
grad_init = compute_gradient(X, y, w_init)

print("\nInitial gradient:")
print(grad_init.T)

# Closed-form solution
w_optimal = np.linalg.inv(X.T @ X) @ X.T @ y

print("\nOptimal w:")
print(w_optimal.T)
print("\nTrue w:")
print(w_true.T)

# 최적해에서 gradient는 0이어야 함
grad_optimal = compute_gradient(X, y, w_optimal)
print("\nGradient at optimal (should be ~0):")
print(grad_optimal.T)

## 3. 자주 헷갈리는 케이스들

### 3.1 Chain Rule과 전치

**문제**: $f(x) = ||Ax - b||^2$의 gradient를 구하라.

**풀이**:

$$f(x) = (Ax - b)^T(Ax - b)$$

$u = Ax - b$로 놓으면:
$$f = u^T u$$

Chain rule:
$$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial u} \frac{\partial u}{\partial x}$$

- $\frac{\partial f}{\partial u} = \frac{\partial (u^T u)}{\partial u} = 2u^T$ (row vector!)
- $\frac{\partial u}{\partial x} = \frac{\partial (Ax)}{\partial x} = A^T$ ← **전치!**

하지만 차원을 맞춰야 합니다:
- $(2u^T) \cdot A^T$는 차원이 안 맞습니다!
- 올바른 형태: $\frac{\partial f}{\partial x} = A^T (2u) = 2A^T(Ax - b)$

**결론**: $\frac{\partial ||Ax - b||^2}{\partial x} = 2A^T(Ax - b)$

In [ ]:
# ||Ax - b||^2의 gradient 검증

A = np.array([[1, 2],
              [3, 4],
              [5, 6]])
x = np.array([[1],
              [1]])
b = np.array([[1],
              [2],
              [3]])

# Loss 계산
residual = A @ x - b
loss = np.sum(residual ** 2)
print("Loss ||Ax - b||^2 =", loss)

# Analytical gradient
grad_analytical = 2 * A.T @ residual
print("\nAnalytical gradient:")
print(grad_analytical.T)

# Numerical gradient (유한 차분)
eps = 1e-5
grad_numerical = np.zeros_like(x)

for i in range(len(x)):
    x_plus = x.copy()
    x_plus[i] += eps
    loss_plus = np.sum((A @ x_plus - b) ** 2)
    
    x_minus = x.copy()
    x_minus[i] -= eps
    loss_minus = np.sum((A @ x_minus - b) ** 2)
    
    grad_numerical[i] = (loss_plus - loss_minus) / (2 * eps)

print("\nNumerical gradient:")
print(grad_numerical.T)

print("\nDifference (should be very small):")
print(np.max(np.abs(grad_analytical - grad_numerical)))

### 3.2 Softmax와 Cross-Entropy의 Gradient

**Softmax**: $\sigma(z)_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$

**Cross-Entropy Loss**: $L = -\sum_i y_i \log \sigma(z)_i$

**Gradient**: $\frac{\partial L}{\partial z} = \sigma(z) - y$

이 유도는 다음 노트북(확률론)에서 자세히 다룹니다!

**핵심**: Softmax + Cross-Entropy를 합치면 gradient가 **매우 간단해집니다**.

In [ ]:
# Softmax + Cross-Entropy gradient

def softmax(z):
    """Numerically stable softmax"""
    exp_z = np.exp(z - np.max(z))  # overflow 방지
    return exp_z / np.sum(exp_z)

def cross_entropy(y_true, y_pred):
    """Cross-entropy loss"""
    return -np.sum(y_true * np.log(y_pred + 1e-10))  # log(0) 방지

# 예제
z = np.array([1.0, 2.0, 3.0])  # logits
y_true = np.array([0, 0, 1])   # one-hot encoded label

# Forward pass
y_pred = softmax(z)
loss = cross_entropy(y_true, y_pred)

print("Logits z:", z)
print("Softmax σ(z):", y_pred)
print("Loss:", loss)

# Gradient: σ(z) - y
grad = y_pred - y_true
print("\nGradient dL/dz:")
print(grad)
print("\n해석: 정답 클래스(index 2)에서 gradient가 음수 → z[2]를 증가시켜야 함")

## 4. 행렬에 대한 미분

### 4.1 Trace Trick

**Trace의 성질**:
- $\text{tr}(A + B) = \text{tr}(A) + \text{tr}(B)$
- $\text{tr}(AB) = \text{tr}(BA)$ (cyclic property)
- $\text{tr}(A^T) = \text{tr}(A)$
- $x^T A x = \text{tr}(x^T A x) = \text{tr}(A x x^T)$

### 4.2 행렬 미분 공식

$$\frac{\partial \text{tr}(AB)}{\partial A} = B^T$$

$$\frac{\partial \text{tr}(A^T B)}{\partial A} = B$$

$$\frac{\partial \log |A|}{\partial A} = (A^{-1})^T = (A^T)^{-1}$$

In [ ]:
# Trace 성질 검증

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
x = np.array([[1], [2]])

# tr(AB) = tr(BA)
print("tr(AB) =", np.trace(A @ B))
print("tr(BA) =", np.trace(B @ A))

# x^T A x = tr(A x x^T)
quad_form = x.T @ A @ x
trace_form = np.trace(A @ x @ x.T)

print("\nx^T A x =", quad_form[0, 0])
print("tr(A x x^T) =", trace_form)

# outer product x x^T
print("\nx x^T (outer product):")
print(x @ x.T)

## 5. 연습문제

### 문제 1: 기본 공식

다음 gradient를 구하시오 ($x \in \mathbb{R}^n$, $A \in \mathbb{R}^{n \times n}$, $b \in \mathbb{R}^n$):

1. $\frac{\partial (b^T x)}{\partial x}$
2. $\frac{\partial (x^T x)}{\partial x}$
3. $\frac{\partial (Ax + b)^T (Ax + b)}{\partial x}$

In [ ]:
# 문제 1 풀이

# 답안:
# 1. d(b^T x)/dx = b
# 2. d(x^T x)/dx = 2x
# 3. d((Ax + b)^T (Ax + b))/dx = 2A^T(Ax + b)

# 검증
n = 3
A = np.random.randn(n, n)
b = np.random.randn(n, 1)
x = np.random.randn(n, 1)

print("=== 문제 1-1: d(b^T x)/dx ===")
# Analytical
grad1_analytical = b
print("Analytical:", grad1_analytical.T)

# Numerical
eps = 1e-5
grad1_numerical = np.zeros_like(x)
for i in range(n):
    x_plus = x.copy()
    x_plus[i] += eps
    f_plus = b.T @ x_plus
    
    x_minus = x.copy()
    x_minus[i] -= eps
    f_minus = b.T @ x_minus
    
    grad1_numerical[i] = (f_plus - f_minus) / (2 * eps)

print("Numerical:", grad1_numerical.T)
print("Error:", np.max(np.abs(grad1_analytical - grad1_numerical)))

print("\n=== 문제 1-2: d(x^T x)/dx ===")
# Analytical
grad2_analytical = 2 * x
print("Analytical:", grad2_analytical.T)

# Numerical
grad2_numerical = np.zeros_like(x)
for i in range(n):
    x_plus = x.copy()
    x_plus[i] += eps
    f_plus = x_plus.T @ x_plus
    
    x_minus = x.copy()
    x_minus[i] -= eps
    f_minus = x_minus.T @ x_minus
    
    grad2_numerical[i] = (f_plus - f_minus) / (2 * eps)

print("Numerical:", grad2_numerical.T)
print("Error:", np.max(np.abs(grad2_analytical - grad2_numerical)))

print("\n=== 문제 1-3: d((Ax+b)^T(Ax+b))/dx ===")
# Analytical
grad3_analytical = 2 * A.T @ (A @ x + b)
print("Analytical:", grad3_analytical.T)

# Numerical
grad3_numerical = np.zeros_like(x)
for i in range(n):
    x_plus = x.copy()
    x_plus[i] += eps
    residual_plus = A @ x_plus + b
    f_plus = residual_plus.T @ residual_plus
    
    x_minus = x.copy()
    x_minus[i] -= eps
    residual_minus = A @ x_minus + b
    f_minus = residual_minus.T @ residual_minus
    
    grad3_numerical[i] = (f_plus - f_minus) / (2 * eps)

print("Numerical:", grad3_numerical.T)
print("Error:", np.max(np.abs(grad3_analytical - grad3_numerical)))

### 문제 2: Logistic Regression

Logistic regression의 loss function:

$$L(w) = -\frac{1}{m}\sum_{i=1}^{m} [y_i \log(\sigma(w^T x_i)) + (1-y_i)\log(1 - \sigma(w^T x_i))]$$

where $\sigma(z) = \frac{1}{1 + e^{-z}}$

**과제**: $\frac{\partial L}{\partial w}$를 유도하고 코드로 검증하시오.

**힌트**: $\frac{d\sigma(z)}{dz} = \sigma(z)(1 - \sigma(z))$

In [ ]:
# 문제 2 풀이

def sigmoid(z):
    """Sigmoid function"""
    return 1 / (1 + np.exp(-z))

def logistic_loss(X, y, w):
    """Logistic regression loss (binary cross-entropy)"""
    m = len(y)
    z = X @ w
    h = sigmoid(z)
    
    # Numerical stability
    loss = -np.mean(y * np.log(h + 1e-10) + (1 - y) * np.log(1 - h + 1e-10))
    return loss

def logistic_gradient(X, y, w):
    """Gradient of logistic regression loss
    
    유도:
    dL/dw = -1/m * Σ [y_i * (1/σ(z_i)) * σ(z_i)(1-σ(z_i)) * x_i - (1-y_i) * (1/(1-σ(z_i))) * σ(z_i)(1-σ(z_i)) * x_i]
          = -1/m * Σ [y_i * (1-σ(z_i)) - (1-y_i) * σ(z_i)] * x_i
          = -1/m * Σ [y_i - σ(z_i)] * x_i
          = 1/m * X^T (σ(Xw) - y)
    """
    m = len(y)
    z = X @ w
    h = sigmoid(z)
    grad = (1 / m) * X.T @ (h - y)
    return grad

# 데이터 생성
np.random.seed(42)
m, n = 100, 3
X = np.random.randn(m, n)
w_true = np.array([[1], [-2], [0.5]])
y = (sigmoid(X @ w_true) > 0.5).astype(float)

w_test = np.random.randn(n, 1) * 0.1

# Analytical gradient
grad_analytical = logistic_gradient(X, y, w_test)
print("Analytical gradient:")
print(grad_analytical.T)

# Numerical gradient
eps = 1e-5
grad_numerical = np.zeros_like(w_test)

for i in range(n):
    w_plus = w_test.copy()
    w_plus[i] += eps
    loss_plus = logistic_loss(X, y, w_plus)
    
    w_minus = w_test.copy()
    w_minus[i] -= eps
    loss_minus = logistic_loss(X, y, w_minus)
    
    grad_numerical[i] = (loss_plus - loss_minus) / (2 * eps)

print("\nNumerical gradient:")
print(grad_numerical.T)

print("\nError:")
print(np.max(np.abs(grad_analytical - grad_numerical)))

### 문제 3: Neural Network Backpropagation

2-layer neural network:

$$h = \sigma(W_1 x + b_1)$$
$$\hat{y} = W_2 h + b_2$$
$$L = \frac{1}{2}||\hat{y} - y||^2$$

**과제**: $\frac{\partial L}{\partial W_1}$, $\frac{\partial L}{\partial W_2}$를 구하시오.

In [ ]:
# 문제 3 풀이

# 네트워크 구조
input_dim = 3
hidden_dim = 4
output_dim = 2

# 파라미터 초기화
np.random.seed(42)
W1 = np.random.randn(hidden_dim, input_dim) * 0.1
b1 = np.zeros((hidden_dim, 1))
W2 = np.random.randn(output_dim, hidden_dim) * 0.1
b2 = np.zeros((output_dim, 1))

# 샘플 데이터
x = np.random.randn(input_dim, 1)
y = np.random.randn(output_dim, 1)

def forward(x, W1, b1, W2, b2):
    """Forward pass"""
    z1 = W1 @ x + b1
    h = sigmoid(z1)
    z2 = W2 @ h + b2
    y_hat = z2
    return z1, h, z2, y_hat

def mse_loss(y_hat, y):
    """MSE loss"""
    return 0.5 * np.sum((y_hat - y) ** 2)

# Forward pass
z1, h, z2, y_hat = forward(x, W1, b1, W2, b2)
loss = mse_loss(y_hat, y)

print("Forward pass:")
print("z1 shape:", z1.shape)
print("h shape:", h.shape)
print("y_hat shape:", y_hat.shape)
print("Loss:", loss)

# Backpropagation (analytical)
# dL/dy_hat = y_hat - y
dL_dy_hat = y_hat - y

# dL/dW2 = dL/dy_hat * dy_hat/dW2 = dL/dy_hat * h^T
dL_dW2 = dL_dy_hat @ h.T

# dL/dh = W2^T @ dL/dy_hat
dL_dh = W2.T @ dL_dy_hat

# dL/dz1 = dL/dh * dh/dz1 = dL/dh * σ'(z1)
dh_dz1 = h * (1 - h)  # sigmoid derivative
dL_dz1 = dL_dh * dh_dz1

# dL/dW1 = dL/dz1 @ x^T
dL_dW1 = dL_dz1 @ x.T

print("\nAnalytical gradients:")
print("dL/dW1 shape:", dL_dW1.shape)
print("dL/dW2 shape:", dL_dW2.shape)

# Numerical gradient for W2
print("\n=== Verifying dL/dW2 ===")
eps = 1e-5
dL_dW2_numerical = np.zeros_like(W2)

for i in range(W2.shape[0]):
    for j in range(W2.shape[1]):
        W2_plus = W2.copy()
        W2_plus[i, j] += eps
        _, _, _, y_hat_plus = forward(x, W1, b1, W2_plus, b2)
        loss_plus = mse_loss(y_hat_plus, y)
        
        W2_minus = W2.copy()
        W2_minus[i, j] -= eps
        _, _, _, y_hat_minus = forward(x, W1, b1, W2_minus, b2)
        loss_minus = mse_loss(y_hat_minus, y)
        
        dL_dW2_numerical[i, j] = (loss_plus - loss_minus) / (2 * eps)

print("Analytical dL/dW2:")
print(dL_dW2)
print("\nNumerical dL/dW2:")
print(dL_dW2_numerical)
print("\nError:", np.max(np.abs(dL_dW2 - dL_dW2_numerical)))

# Sample check for W1 (only first element for speed)
print("\n=== Verifying dL/dW1[0,0] ===")
W1_plus = W1.copy()
W1_plus[0, 0] += eps
_, _, _, y_hat_plus = forward(x, W1_plus, b1, W2, b2)
loss_plus = mse_loss(y_hat_plus, y)

W1_minus = W1.copy()
W1_minus[0, 0] -= eps
_, _, _, y_hat_minus = forward(x, W1_minus, b1, W2, b2)
loss_minus = mse_loss(y_hat_minus, y)

dL_dW1_00_numerical = (loss_plus - loss_minus) / (2 * eps)

print("Analytical dL/dW1[0,0]:", dL_dW1[0, 0])
print("Numerical dL/dW1[0,0]:", dL_dW1_00_numerical)
print("Error:", abs(dL_dW1[0, 0] - dL_dW1_00_numerical))

## 6. 요약 및 체크리스트

### 핵심 공식 정리

| 함수 | Gradient | 비고 |
|------|----------|------|
| $Ax$ | $A^T$ | **전치 필요!** |
| $x^T A$ | $A$ | 전치 없음 |
| $x^T A x$ (A 대칭) | $2Ax$ | 이차 형식 |
| $x^T A x$ (A 일반) | $(A + A^T)x$ | 일반 이차 형식 |
| $\|Ax - b\|^2$ | $2A^T(Ax - b)$ | MSE의 기본 |

### 전치가 나타나는 이유

1. **Chain rule**: $\frac{\partial f}{\partial x} = \frac{\partial f}{\partial y} \frac{\partial y}{\partial x}$에서 Jacobian의 전치
2. **차원 일치**: Gradient는 항상 원래 변수와 같은 shape
3. **Denominator layout**: ML에서 표준 규약

### 자주 하는 실수

- ❌ $\frac{\partial (Ax)}{\partial x} = A$ (틀림!)
- ✅ $\frac{\partial (Ax)}{\partial x} = A^T$ (맞음!)

- ❌ Chain rule에서 차원 안 맞춤
- ✅ 항상 차원 체크하기

### 다음 단계

- 다음 노트북: **확률론 심화** (조건부 확률 → Bayes → Softmax)
- 추가 학습: ML_L04_vector.calculus_review.pdf

## 7. 추가 연습문제

직접 풀어보고 numerical gradient로 검증하세요!

1. $\frac{\partial ||x||_2}{\partial x}$ (L2 norm)
2. $\frac{\partial (x^T A^T A x)}{\partial x}$
3. $\frac{\partial \log(1 + e^{w^T x})}{\partial w}$ (logistic loss의 일부)
4. Ridge regression loss: $L(w) = ||Xw - y||^2 + \lambda ||w||^2$의 gradient
5. Weighted MSE: $L(w) = (Xw - y)^T W (Xw - y)$의 gradient (W는 대칭 행렬)

In [ ]:
# 추가 연습문제 템플릿
# 여기에 직접 풀어보세요!

# 문제 1: d(||x||_2)/dx
# 답: x / ||x||_2

# 문제 2: d(x^T A^T A x)/dx  
# 답: 2 A^T A x (A^T A는 대칭)

# 문제 3: d(log(1 + e^(w^T x)))/dw
# 답: σ(w^T x) * x where σ is sigmoid

# 문제 4: Ridge regression gradient
# 답: 2 X^T(Xw - y) + 2λw

# 문제 5: Weighted MSE gradient
# 답: 2 X^T W (Xw - y)